1. Write data to delta lake (managed table)
1. Write data to delta lake (external table)
1. Read data from delta lake (Table)
1. Read data from delta lake (File)

In [0]:
%sql
CREATE DATABASE IF NOT EXISTS f1_demo
LOCATION '/mnt/formula1stg/demo'

In [0]:
%python
results_df = spark.read \
.option("inferSchema", True) \
.json("/mnt/formula1stg/raw/2021-03-28/results.json")


In [0]:
%python
results_df.write.format("delta").mode("overwrite").saveAsTable("f1_demo.results_managed")

In [0]:
%sql
SELECT * FROM f1_demo.results_managed

In [0]:
%python
results_df.write.format("delta").mode("overwrite").save("/mnt/formula1stg/demo/results_external")

In [0]:
%sql
CREATE TABLE IF NOT EXISTS f1_demo.results_external
USING DELTA
LOCATION '/mnt/formula1stg/demo/results_external'

In [0]:
%sql
SELECT * FROM f1_demo.results_external;

In [0]:
%python
results_external_df = spark.read.format("delta").load("/mnt/formula1stg/demo/results_external")

In [0]:
%python
display(results_external_df)

In [0]:
%python
results_df.write.format("delta").mode("overwrite").partitionBy("constructorid").saveAsTable("f1_demo.results_partitioned")

In [0]:
%sql
SHOW PARTITIONS f1_demo.results_partitioned

1. Update Delta Table
1. Delete From Delta Table

In [0]:
%sql
SELECT * FROM f1_demo.results_managed

In [0]:
%sql
UPDATE f1_demo.results_managed
  SET points = 11 - position
  WHERE position <= 10

In [0]:
-- UPDATE f1_demo.results_managed
--   SET position = 20
--   WHERE number = 9

In [0]:
%sql
DELETE FROM f1_demo.results_managed
WHERE POSITION > 10;

In [0]:
%sql
SELECT * FROM f1_demo.results_managed

In [0]:
%python
from delta.tables import *

deltaTable = DeltaTable.forPath(spark, '/mnt/formula1stg/demo/results_managed')

# Declare the predicate by using a SQL-formatted string.
deltaTable.delete("points =0")

##### Upsert using merge

In [0]:
%python
drivers_day1_df = spark.read \
.option("inferSchema", "true") \
.json("/mnt/formula1stg/raw/2021-03-28/drivers.json") \
.filter("driverId <=10") \
.select("driverId", "dob", "name.forename", "name.surname")

In [0]:
%python
display(drivers_day1_df)

In [0]:
%python
drivers_day1_df.createOrReplaceTempView("drivers_day1")

In [0]:
%python
from pyspark.sql.functions import upper

drivers_day2_df = spark.read \
.option("inferSchema", "true") \
.json("/mnt/formula1stg/raw/2021-03-28/drivers.json") \
.filter("driverId between 6 AND 15") \
.select("driverId", "dob", upper("name.forename").alias("forename"), upper("name.surname").alias("surname"))


In [0]:
%python
display(drivers_day2_df)

In [0]:
%python
drivers_day2_df.createOrReplaceTempView("drivers_day2")

In [0]:
%python
from pyspark.sql.functions import upper

drivers_day3_df = spark.read \
.option("inferSchema", "true") \
.json("/mnt/formula1stg/raw/2021-03-28/drivers.json") \
.filter("driverId between 1 AND 5 OR driverId between 16 AND 20") \
.select("driverId", "dob", upper("name.forename").alias("forename"), upper("name.surname").alias("surname"))


In [0]:
%python
display(drivers_day3_df)

In [0]:
CREATE TABLE IF NOT EXISTS f1_demo.drivers_merge (
  driverId INT,
  dob DATE,
  forename STRING,
  surname STRING,
  createdDate DATE,
  updatedDate DATE
)
USING DELTA

#### Day 1

In [0]:
MERGE INTO f1_demo.drivers_merge tgt
USING drivers_day1 upd
ON tgt.driverId = upd.driverId
WHEN MATCHED THEN 
  UPDATE SET
    tgt.dob = upd.dob,
    tgt.forename = upd.forename,
    tgt.surname = upd.surname,
    tgt.updatedDate = current_timestamp()
WHEN NOT MATCHED
  THEN INSERT ( driverId, dob, forename, surname, createdDate)  
  VALUES (driverId, dob, forename, surname, current_timestamp())



In [0]:

SELECT * FROM f1_demo.drivers_merge

#### Day 2

In [0]:
MERGE INTO f1_demo.drivers_merge tgt
USING drivers_day2 upd
ON tgt.driverId = upd.driverId
WHEN MATCHED THEN 
  UPDATE SET
    tgt.dob = upd.dob,
    tgt.forename = upd.forename,
    tgt.surname = upd.surname,
    tgt.updatedDate = current_timestamp()
WHEN NOT MATCHED
  THEN INSERT ( driverId, dob, forename, surname, createdDate)  
  VALUES (driverId, dob, forename, surname, current_timestamp())

In [0]:

SELECT * FROM f1_demo.drivers_merge;

In [0]:
%python
from pyspark.sql.functions import current_timestamp
from delta.tables import DeltaTable

deltaTable = DeltaTable.forPath(spark, "/mnt/formula1stg/demo/drivers_merge")

deltaTable.alias("tgt").merge(
    drivers_day3_df.alias("upd"),
    "tgt.driverId = upd.driverId"
).whenMatchedUpdate(
    set={
        "dob": "upd.dob",
        "forename": "upd.forename",
        "surname": "upd.surname",
        "updatedDate": current_timestamp()
    }
).whenNotMatchedInsert(
    values={
        "driverId": "upd.driverId",
        "dob": "upd.dob",
        "forename": "upd.forename",
        "surname": "upd.surname",
        "createdDate": current_timestamp()
    }
).execute()

In [0]:
SELECT * FROM f1_demo.drivers_merge
--ORDER by driverId

In [0]:
--DROP TABLE f1_demo.drivers_merge

### History, Time Travel and Vacuum

In [0]:
DESC HISTORY f1_demo.drivers_merge

In [0]:
SELECT * FROM f1_demo.drivers_merge
VERSION AS OF 1

In [0]:
SELECT * FROM f1_demo.drivers_merge TIMESTAMP AS OF '2025-10-30T20:20:03.000+00:00';

In [0]:
%python
df = spark.read.format("delta").option("timestampAsOf", '2025-10-30T20:20:03.000+00:00').load("/mnt/formula1stg/demo/drivers_merge")
display(df)

In [0]:
-- REMOVING DATA FROM A TABLE WITH VACUUM so it is removed completely. 
--The below command will remove the file
--SET spark.databricks.delta.retentionDurationCheck.enabled = false
--VACUUM f1_demo.drivers_merge RETAIN 0 HOURS

In [0]:
DELETE FROM f1_demo.drivers_merge WHERE driverId = 1;

In [0]:
SELECT * FROM f1_demo.drivers_merge;

In [0]:
SELECT * FROM f1_demo.drivers_merge VERSION AS OF 4;

In [0]:
--RECOVEING DELETED DATA
MERGE INTO f1_demo.drivers_merge tgt
USING f1_demo.drivers_merge VERSION AS OF 4 src
ON tgt.driverId = src.driverId
WHEN NOT MATCHED THEN INSERT *

In [0]:
DESC HISTORY f1_demo.drivers_merge

In [0]:
SELECT * FROM f1_demo.drivers_merge

### Transaction Logs

In [0]:
CREATE TABLE IF NOT EXISTS f1_demo.drivers_txn (
  driverId INT,
  dob DATE,
  forename STRING,
  surname STRING,
  createdDate DATE,
  updatedDate DATE
)
USING DELTA

In [0]:
DESC HISTORY f1_demo.drivers_txn

In [0]:
INSERT INTO f1_demo.drivers_txn
SELECT * FROM f1_demo.drivers_merge
WHERE driverId = 1 

In [0]:
DESC HISTORY f1_demo.drivers_txn

In [0]:
DELETE FROM f1_demo.drivers_txn
WHERE driverId = 1

In [0]:
%python
for driver_id in range(2, 20):
    spark.sql(f"""INSERT INTO f1_demo.drivers_txn
        SELECT * FROM f1_demo.drivers_merge
        WHERE driverId = {driver_id}""")

In [0]:
---  Go to storage explorer to view history logs from the delta_log folder of the table
--- Transaction logs are kept for only 30 days
--- Vacuum removes all the data

#### Convert Parquet to Delta

In [0]:
CREATE TABLE IF NOT EXISTS f1_demo.drivers_convert_to_delta (
  driverId INT,
  dob DATE,
  forename STRING,
  surname STRING,
  createdDate DATE,
  updatedDate DATE
)
USING PARQUET 


In [0]:
INSERT INTO f1_demo.drivers_convert_to_delta
SELECT * FROM f1_demo.drivers_merge

In [0]:
CONVERT TO DELTA f1_demo.drivers_convert_to_delta

####When Using Parquet file

In [0]:
%python
df = spark.table("f1_demo.drivers_convert_to_delta")

In [0]:
df.write.format("parquet").save("/mnt/formula1stg/demo/drivers_convert_to_delta_new")

In [0]:
CONVERT TO DELTA parquet.`/mnt/formula1stg/demo/drivers_convert_to_delta_new`